# Simulation Study for Noise Filtering

This is the v2 for the simulation study on the sparse jump model comparison with HMM, to show that SJM is able to filter away noisy data by using the weighting in the algorithm.


In [2]:
#Load Packages
import numpy as np
import pandas as pd
from hmmlearn.hmm import GaussianHMM  # Import GaussianHMM
from sklearn.metrics import balanced_accuracy_score, confusion_matrix
from scipy.optimize import linear_sum_assignment
from jumpmodels.sparse_jump import SparseJumpModel
from jumpmodels.jump import JumpModel
from scipy import stats
from joblib import Parallel, delayed
import multiprocessing
from scipy.stats import wilcoxon
from jumpmodels.preprocess import StandardScalerPD, DataClipperStd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans


## 1. Data Simulation & Utility Functions
def simulate_data(T, P, mu, random_state=None): """ Simulate data from a 2-state Gaussian HMM with correlation (tbd) between the features

In [ ]:
# 1. Data simulation
def simulate_data(T, P, mu, random_state=None):
    """
    Simulate data from a 2-state Gaussian HMM with correlated noise for features beyond
    the first 15 informative features.
    """
    rng = np.random.default_rng(random_state)
    
    # Transition matrix for 2 states
    transmat = np.array([[0.9980, 0.0020],
                         [0.0114, 0.9886]])
    
    # Compute stationary distribution
    eigvals, eigvecs = np.linalg.eig(transmat.T)
    stat = np.real(eigvecs[:, np.isclose(eigvals, 1)])
    stat = stat[:, 0]
    stat = stat / np.sum(stat)
    
    # Generate state sequence
    states = np.zeros(T, dtype=int)
    states[0] = rng.choice(np.arange(2), p=stat)
    for t in range(1, T):
        states[t] = rng.choice(np.arange(2), p=transmat[states[t-1]])
    
    # Define means for each state: first 15 features are informative
    means = np.zeros((2, P))
    if P >= 15:
        means[0, :15] = -mu
        means[1, :15] = mu
    else:
        means[0, :P] = -mu
        means[1, :P] = mu
    
    # Prepare correlated noise for features beyond the first 15
    informative = 15
    if P > informative:
        num_noise = P - informative
        sigma = np.full((num_noise, num_noise), 0.00)
        np.fill_diagonal(sigma, 1.0)
        C = np.linalg.cholesky(sigma)
    else:
        C = None
    
    # Generate observations
    X = np.zeros((T, P))
    for t in range(T):
        n_inf = min(P, informative)
        X[t, :n_inf] = rng.normal(loc=means[states[t], :n_inf], scale=1.0, size=n_inf)
        if P > informative:
            noise_indep = rng.normal(loc=0.0, scale=1.0, size=P - informative)
            X[t, informative:] = C @ noise_indep
    return X, states

## 2.Aligning Predicted Labels With True Labels using the Hungarian Algorithm

In [4]:
def align_labels(true_labels, pred_labels):
    """
    Align predicted labels with true labels using the Hungarian algorithm.
    """
    D = confusion_matrix(true_labels, pred_labels)
    row_ind, col_ind = linear_sum_assignment(-D)
    mapping = {col: row for row, col in zip(row_ind, col_ind)}
    aligned = np.array([mapping[x] for x in pred_labels])
    return aligned

## 3. Setting up the function to calcuate the BAC

In [5]:
def calculate_bac(true_states, pred_states):
    """
    Compute balanced accuracy score after aligning predicted state labels.
    """
    aligned_pred = align_labels(true_states, pred_states)
    return balanced_accuracy_score(true_states, aligned_pred)

## 3.1 Computing per state accuracy and calculating statistic

In [6]:
def compute_per_state_accuracy(true_states, pred_states):
    """
    Compute per-state accuracy and BAC.
    Returns:
        acc1: accuracy for state 0,
        acc2: accuracy for state 1,
        bac: balanced accuracy.
    """
    aligned = align_labels(true_states, pred_states)
    cm = confusion_matrix(true_states, aligned, labels=[0, 1])
    acc1 = cm[0, 0] / cm[0].sum() if cm[0].sum() > 0 else np.nan
    acc2 = cm[1, 1] / cm[1].sum() if cm[1].sum() > 0 else np.nan
    bac = (acc1 + acc2) / 2
    return acc1, acc2, bac

def compute_state_statistics(X, true_states, pred_states):
    """
    Compute mean for each state.
    Returns a dict with keys: state0_mean, state1_mean.
    """
    aligned = align_labels(true_states, pred_states)
    stats_dict = {}
    for state in [0, 1]:
        idx = np.where(aligned == state)[0]
        if len(idx) > 0:
            stats_dict[f'state{state}_mean'] = np.mean(X[idx])
        else:
            stats_dict[f'state{state}_mean'] = np.nan
    return stats_dict

## 3.2 Computing transition probabilities

In [7]:
def compute_transition_probabilities(state_seq):
    """
    Compute q12 and q21 from the state sequence.
    q12: transitions from state 0 to 1 / total transitions from state 0.
    q21: transitions from state 1 to 0 / total transitions from state 1.
    """
    state_seq = np.array(state_seq)
    prev = state_seq[:-1]
    nxt = state_seq[1:]
    if np.sum(prev==0) > 0:
        q12 = np.sum((prev==0) & (nxt==1)) / np.sum(prev==0)
    else:
        q12 = np.nan
    if np.sum(prev==1) > 0:
        q21 = np.sum((prev==1) & (nxt==0)) / np.sum(prev==1)
    else:
        q21 = np.nan
    return q12, q21

## 3.3 Standardizing features

In [8]:
def standardize_features(X):
    """
    Standardize features using the same procedure as in study 2.
    The function converts X into a DataFrame, applies DataClipperStd with mul=3.0,
    then applies StandardScalerPD (zero mean, unit variance) and returns the standardized array.
    """
    X_df = pd.DataFrame(X)
    clipper = DataClipperStd(mul=3.0)
    scaler = StandardScalerPD()
    X_clipped = clipper.fit_transform(X_df)
    X_scaled = scaler.fit_transform(X_clipped)
    return X_scaled.values

## 4. Functions for model formulation

### 4.1 HMM With Nystrup (2020b) initialization

In [9]:
def run_mle(observations, n_components=2, init_type='default', seed=None):
    """
    Fit a Gaussian HMM to the data.
    init_type: 'default' uses fixed initialization; 'kmeans' uses KMeans initialization.
    For 'default', we use:
      - startprob_ = [1, 0]
      - transmat_ = [[0.9, 0.1], [0.1, 0.9]]
      - Means initialized to [0, 0] and covars to a small positive value.
    """
    model = GaussianHMM(n_components=n_components, covariance_type='diag', n_iter=100, random_state=seed)
    
    if init_type=='default':
        model.startprob_ = np.array([1.0, 0.0])
        model.transmat_ = np.array([[0.9, 0.1],
                                    [0.1, 0.9]])
        model.means_ = np.zeros((n_components, observations.shape[1]))
        # Ensure nonzero covariances:
        model.covars_ = np.full((n_components, observations.shape[1]), 1e-2)
        model.init_params = ''  # do not reinitialize
    elif init_type=='kmeans':
        kmeans = KMeans(n_clusters=n_components, n_init=10, random_state=seed)
        labels = kmeans.fit_predict(observations)
        means = []
        covars = []
        for i in range(n_components):
            obs_i = observations[labels==i]
            means.append(np.mean(obs_i, axis=0))
            covars.append(np.var(obs_i, axis=0) + 1e-2)
        model.startprob_ = np.ones(n_components) / n_components
        model.transmat_ = np.ones((n_components, n_components)) / n_components
        model.means_ = np.array(means)
        model.covars_ = np.array(covars)
        model.init_params = 'tmc'
    model.fit(observations)
    pred_states = model.predict(observations)
    return model, pred_states

# Convenience wrappers:
def run_mle_default(observations, seed=None):
    return run_mle(observations, init_type='default', seed=seed)

def run_mle_kmeans(observations, seed=None):
    return run_mle(observations, init_type='kmeans', seed=seed)

### 4.2 Normal (Standard) Jump Model with Grid Search over λ

In [10]:
def run_jump_model_grid_search(X, true_states, n_components=2, random_state=None):
    """
    Grid search for the jump model over a range of lambda values.
    Returns best predicted states, best BAC, best lambda, and a DataFrame of grid search results.
    """
    lambda_values = np.logspace(-2, 4, 1)
    best_bac = -np.inf
    best_labels = None
    best_lambda = None
    grid_results = []
    for lam in lambda_values:
        model = JumpModel(n_components=n_components, jump_penalty=lam, cont=False,
                          max_iter=10, random_state=random_state)
        model.fit(X)
        labels = model.labels_
        bac = calculate_bac(true_states, labels)
        grid_results.append({'lambda': lam, 'BAC': bac})
        if bac > best_bac:
            best_bac = bac
            best_labels = labels
            best_lambda = lam
    return best_labels, best_bac, best_lambda, pd.DataFrame(grid_results)

### 4.3 Sparse Jump Model with Grid Search over λ and kappa

In [11]:
def run_sparse_jump_model_grid_search(X, true_states, n_components=2, random_state=None):
    """
    Grid search for the sparse jump model over combinations of lambda and kappa.
    Returns best predicted states, best BAC, best lambda, best kappa, and a DataFrame of grid search results.
    """
    lambdas = np.logspace(-1, 2, 1)
    p = X.shape[1]
    kappas = np.linspace(1, np.sqrt(p), 1)
    best_bac = -np.inf
    best_labels = None
    best_lambda = None
    best_kappa = None
    grid_results = []
    for lam in lambdas:
        for kappa in kappas:
            max_feats = kappa**2
            model = SparseJumpModel(n_components=n_components, jump_penalty=lam, cont=False,
                                     max_feats=max_feats, max_iter=10, random_state=random_state)
            model.fit(X)
            labels = model.labels_
            bac = calculate_bac(true_states, labels)
            grid_results.append({'lambda': lam, 'kappa': kappa, 'max_feats': max_feats, 'BAC': bac})
            if bac > best_bac:
                best_bac = bac
                best_labels = labels
                best_lambda = lam
                best_kappa = kappa
    return best_labels, best_bac, best_lambda, best_kappa, pd.DataFrame(grid_results)

# 5. Main Execution
 We split the code into three sections for each model and then combine results at the end.


In [12]:
# Settings
T = 500
mu_values = [0.01, 0.02, 0.05, 0.1, 0.25, 0.5]
p_values = [15, 30, 60, 150, 300]
n_simulations = 1  # adjust as needed
n_components = 2

# Lists to collect simulation details and grid search results
simulation_results = []
grid_search_jump_list = []
grid_search_sparse_list = []

num_cores = multiprocessing.cpu_count()

for mu in mu_values:
    for P in p_values:
        sim_results = []
        # Parallelize simulations for this mu and P combination
        results = Parallel(n_jobs=num_cores)(
            delayed(lambda sim: {
                'sim': sim,
                'X': simulate_data(T, P, mu, random_state=sim)[0],
                'true_states': simulate_data(T, P, mu, random_state=sim)[1]
            })(sim) for sim in range(n_simulations)
        )
        for res in results:
            X = res['X']
            true_states = res['true_states']
            
            # --- MLE default and MLE_k (kmeans) on raw data ---
            model_mle, pred_mle = run_mle_default(X, seed=res['sim'])
            model_mle_k, pred_mle_k = run_mle_kmeans(X, seed=res['sim'])
            bac_mle = calculate_bac(true_states, pred_mle)
            bac_mle_k = calculate_bac(true_states, pred_mle_k)
            
            # --- Standardize features for jump models ---
            X_std = standardize_features(X)
            
            # --- Jump model grid search ---
            pred_jump, bac_jump, best_lambda_jump, df_grid_jump = run_jump_model_grid_search(
                X_std, true_states, n_components=n_components, random_state=res['sim'])
            
            # --- Sparse jump model grid search ---
            pred_sparse, bac_sparse, best_lambda_sparse, best_kappa_sparse, df_grid_sparse = run_sparse_jump_model_grid_search(
                X_std, true_states, n_components=n_components, random_state=res['sim'])
            
            # --- Compute performance metrics ---
            acc1_mle, acc2_mle, _ = compute_per_state_accuracy(true_states, pred_mle)
            acc1_mle_k, acc2_mle_k, _ = compute_per_state_accuracy(true_states, pred_mle_k)
            acc1_jump, acc2_jump, _ = compute_per_state_accuracy(true_states, pred_jump)
            acc1_sparse, acc2_sparse, _ = compute_per_state_accuracy(true_states, pred_sparse)
            
            stats_mle = compute_state_statistics(X, true_states, pred_mle)
            stats_mle_k = compute_state_statistics(X, true_states, pred_mle_k)
            stats_jump = compute_state_statistics(X, true_states, pred_jump)
            stats_sparse = compute_state_statistics(X, true_states, pred_sparse)
            
            q12_mle, q21_mle = compute_transition_probabilities(pred_mle)
            q12_mle_k, q21_mle_k = compute_transition_probabilities(pred_mle_k)
            q12_jump, q21_jump = compute_transition_probabilities(pred_jump)
            q12_sparse, q21_sparse = compute_transition_probabilities(pred_sparse)
            
            # Save all results in a dict for this simulation run (volatility fields removed)
            run_dict = {
                'mu': mu,
                'P': P,
                'sim': res['sim'],
                # MLE (default)
                'MLE_BAC': bac_mle,
                'MLE_acc1': acc1_mle,
                'MLE_acc2': acc2_mle,
                'MLE_state0_mean': stats_mle.get('state0_mean', np.nan),
                'MLE_state1_mean': stats_mle.get('state1_mean', np.nan),
                'MLE_q12': q12_mle,
                'MLE_q21': q21_mle,
                # MLE_k (kmeans)
                'MLEK_BAC': bac_mle_k,
                'MLEK_acc1': acc1_mle_k,
                'MLEK_acc2': acc2_mle_k,
                'MLEK_state0_mean': stats_mle_k.get('state0_mean', np.nan),
                'MLEK_state1_mean': stats_mle_k.get('state1_mean', np.nan),
                'MLEK_q12': q12_mle_k,
                'MLEK_q21': q21_mle_k,
                # Jump Model
                'Jump_BAC': bac_jump,
                'Jump_acc1': acc1_jump,
                'Jump_acc2': acc2_jump,
                'Jump_state0_mean': stats_jump.get('state0_mean', np.nan),
                'Jump_state1_mean': stats_jump.get('state1_mean', np.nan),
                'Jump_q12': q12_jump,
                'Jump_q21': q21_jump,
                'Jump_best_lambda': best_lambda_jump,
                # Sparse Jump Model
                'SparseJump_BAC': bac_sparse,
                'SparseJump_acc1': acc1_sparse,
                'SparseJump_acc2': acc2_sparse,
                'SparseJump_state0_mean': stats_sparse.get('state0_mean', np.nan),
                'SparseJump_state1_mean': stats_sparse.get('state1_mean', np.nan),
                'SparseJump_q12': q12_sparse,
                'SparseJump_q21': q21_sparse,
                'SparseJump_best_lambda': best_lambda_sparse,
                'SparseJump_best_kappa': best_kappa_sparse
            }
            sim_results.append(run_dict)
            
            # Save grid search results with additional identifiers
            df_grid_jump['mu'] = mu
            df_grid_jump['P'] = P
            df_grid_jump['sim'] = res['sim']
            grid_search_jump_list.append(df_grid_jump)
            
            df_grid_sparse['mu'] = mu
            df_grid_sparse['P'] = P
            df_grid_sparse['sim'] = res['sim']
            grid_search_sparse_list.append(df_grid_sparse)
            
        simulation_results.extend(sim_results)

# Final DataFrames
df_simulation = pd.DataFrame(simulation_results)
df_grid_jump = pd.concat(grid_search_jump_list, ignore_index=True)
df_grid_sparse = pd.concat(grid_search_sparse_list, ignore_index=True)

print("Simulation results shape:", df_simulation.shape)
print("Jump grid search results shape:", df_grid_jump.shape)
print("Sparse Jump grid search results shape:", df_grid_sparse.shape)

Even though the 'transmat_' attribute is set, it will be overwritten during initialization because 'init_params' contains 't'
Even though the 'means_' attribute is set, it will be overwritten during initialization because 'init_params' contains 'm'
Even though the 'covars_' attribute is set, it will be overwritten during initialization because 'init_params' contains 'c'
/Users/vlad/Desktop/git/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:2480: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/Users/vlad/Desktop/git/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:2480: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/Users/vlad/Desktop/git/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:2480: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in 

Simulation results shape: (30, 34)
Jump grid search results shape: (30, 5)
Sparse Jump grid search results shape: (30, 7)


/Users/vlad/Desktop/git/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:2480: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


In [16]:
df_simulation

,mu,P,sim,MLE_BAC,MLE_acc1,MLE_acc2,MLE_state0_mean,MLE_state1_mean,MLE_q12,MLE_q21,...,Jump_best_lambda,SparseJump_BAC,SparseJump_acc1,SparseJump_acc2,SparseJump_state0_mean,SparseJump_state1_mean,SparseJump_q12,SparseJump_q21,SparseJump_best_lambda,SparseJump_best_kappa
0,0.01,15,0,0.636,0.636,NaN,-0.007179,-0.011191,0.027473,0.012618,...,0.01,0.548,0.548,NaN,0.047896,-0.077182,0.412409,0.497778,0.1,1.0
1,0.01,30,0,0.604,0.604,NaN,0.169958,-0.254965,0.394040,0.598985,...,0.01,0.506,0.506,NaN,0.127382,-0.127058,0.503968,0.514170,0.1,1.0
2,0.01,60,0,0.548,0.548,NaN,-0.222709,0.325252,0.601770,0.494505,...,0.01,0.572,0.572,NaN,0.160831,-0.156602,0.416084,0.553991,0.1,1.0
3,0.01,150,0,0.532,0.532,NaN,-0.297213,0.323442,0.532189,0.466165,...,0.01,0.522,0.522,NaN,0.133215,-0.159592,0.446154,0.489540,0.1,1.0
4,0.01,300,0,0.504,0.504,NaN,-0.319631,0.309280,0.500000,0.490040,...,0.01,0.534,0.534,NaN,0.143060,-0.180440,0.468165,0.538793,0.1,1.0
5,0.02,15,0,0.636,0.636,NaN,-0.017179,-0.021191,0.027473,0.012618,...,0.01,0.548,0.548,NaN,0.037896,-0.087182,0.412409,0.497778,0.1,1.0
6,0.02,30,0,0.604,0.604,NaN,0.164958,-0.259965,0.394040,0.598985,...,0.01,0.506,0.506,NaN,0.122382,-0.132058,0.503968,0.514170,0.1,1.0
7,0.02,60,0,0.548,0.548,NaN,-0.225209,0.322752,0.601770,0.494505,...,0.01,0.572,0.572,NaN,0.158331,-0.159102,0.416084,0.553991,0.1,1.0
8,0.02,150,0,0.532,0.532,NaN,-0.298213,0.322442,0.532189,0.466165,...,0.01,0.522,0.522,NaN,0.132215,-0.160592,0.446154,0.489540,0.1,1.0
9,0.02,300,0,0.504,0.504,NaN,-0.320131,0.308780,0.500000,0.490040,...,0.01,0.534,0.534,NaN,0.142560,-0.180940,0.468165,0.538793,0.1,1.0


## 6 Wilxoxon Test Table Generation

In [ ]:
# Create a Wilcoxon table comparing Sparse Jump BAC versus the other models (MLE, MLEK, Jump)
wilcoxon_rows = []

for mu in mu_values:
    for P in p_values:
        df_subset = df_simulation[(df_simulation['mu'] == mu) & (df_simulation['P'] == P)]
        if len(df_subset) < 2:
            continue
        # Compare BAC: Sparse Jump vs MLE
        stat, p_sparse_mle = wilcoxon(df_subset['SparseJump_BAC'], df_subset['MLE_BAC'])
        # Compare BAC: Sparse Jump vs MLE_k
        stat, p_sparse_mlek = wilcoxon(df_subset['SparseJump_BAC'], df_subset['MLEK_BAC'])
        # Compare BAC: Sparse Jump vs Jump
        stat, p_sparse_jump = wilcoxon(df_subset['SparseJump_BAC'], df_subset['Jump_BAC'])
        
        wilcoxon_rows.append({
            'mu': mu,
            'P': P,
            'p_value_SparseJump_vs_MLE': p_sparse_mle,
            'p_value_SparseJump_vs_MLEK': p_sparse_mlek,
            'p_value_SparseJump_vs_Jump': p_sparse_jump
        })

df_wilcoxon = pd.DataFrame(wilcoxon_rows)
print("Wilcoxon test results (Sparse Jump vs other models):")
print(df_wilcoxon)
